In [ ]:
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly
plotly.offline.init_notebook_mode()
from toolkit.tire_model.tire_model_pacejka_2010 import *
from toolkit.tire_model.tire_model_fitting import *
from toolkit.tire_model.ttc_loader import *
from toolkit.tire_model.tire_fitting_masks import *
from toolkit.tire_model.tire_model_utils import *
from toolkit.common.constants import *

In [ ]:
# Get the ttc data for the tire you want to fit
combi_runs, cornering, drive_brake, name = load_runs(get_R20_18x6_7_runs())
# create all the boring lists and stuff
params_list: List = []
error_list: List = []

In [ ]:
# Let's just take a leap of fath and ignore this for a second, we'll come back to it
tire_model = tire_model_from_arr(H_R20_18X6_7)
fz_nom = tire_model.FNOMIN
pres_nom = tire_model.NOMPRES
# PHY1, PHY2, PVY1, PVY2 = 0.0, 0.0, 0.0, 0.0

In [ ]:
def plot_fit(fz_targ, func, sa, ia, pres, kappa, figs, dg, name, fz_err=False, long=False):
    dd = dg[np.abs(dg.FZ - fz_targ) < fz_targ*0.1+20]
    dd = dd[np.abs(dd.IA - ia) < np.deg2rad(0.75)]
    if long:
        dd = dd[np.abs(dd.SA - sa) < np.deg2rad(0.75)]
    dd = filter_press(dd, press=pres, d_press=6000)
    if dd.shape[0] == 0:
        return None
    fy = func(fz_targ, sa, ia, pres, kappa, offsets=True)
    fy_real = func(fz_targ, dd.SA, dd.IA, dd.P, dd.SL)
    shy, svy = None, None
    if type(fy) is tuple:
        shy = fy[1]
        svy = fy[2]
        fy = fy[0]
    error = np.mean(np.abs(fy_real - dd.FY))
    if long:
        xx = dd.SL
        figs.add_trace(go.Scattergl(x=kappa, y=fy, name=f"{name}\t{error:.2f} N rmse", legendgroup=f"{name}"))
    else:
        xx = np.rad2deg(dd.SA)
        figs.add_trace(go.Scattergl(x=np.rad2deg(sa), y=fy, name=f"{name}\t{error:.2f} N rmse", legendgroup=f"{name}"))
    if fz_err:
        fz_err = (dd.FZ - fz_targ)
        figs.add_trace(go.Scattergl(x=xx, y=dd.FZ, mode='markers', marker=dict(size=1.5, color=fz_err, colorscale='Viridis', showscale=True), legendgroup=f"{name}", showlegend=False))
    else:
        figs.add_trace(go.Scattergl(x=xx, y=dd.FY, mode='markers', marker=dict(size=1), legendgroup=f"{name}", showlegend=False))
    if shy is not None:
        if long:
            figs.add_trace(go.Scattergl(x=shy, y=svy, mode='markers', marker=dict(size=4, color='red'), legendgroup=f"{name}", showlegend=False))
        else:
            figs.add_trace(go.Scattergl(x=shy, y=svy, mode='markers', marker=dict(size=4, color='red'), legendgroup=f"{name}", showlegend=False))
    return error
slip_angles = np.linspace(-0.25, 0.25, 100)
kappa_sweep = np.linspace(-0.3, 0.3, 100)
sweeps = [220, 440, 660, 880, 1100]

Alright lets just list off all the equations from where we left off in the last notebook. We have the following equations:<br>
Here is a [link](https://pure.tue.nl/ws/files/3139488/677330157969510.pdf) to a draft of the Besselink paper that is a good reference for the equations below<br>
This paper is for MFTire 6.1, the only difference is that 6.2 includes turnslip.<br>
All we have to do is add pressure and the combined loading and then we will be at the actual 6.1 equations<br>
<br>
$df_z=\frac{F_z-F_{z0}}{F_{z0}}$ eqn. 31<br>
Do note that in the aforementioned draft paper gives the equation $df_z=\frac{F_z-F_{z0}}{F_z}$ which is a typo<br>
$F_{yp}=D_y*sin(C_y*arctan(B_y*\alpha_y-E_y*(B_y*\alpha_y-arctan(B_y*\alpha_y))))+S_{Vy0}$  eqn. 52<br>
$\alpha_y=\alpha_F+S_{Hy0}$ eqn. 53<br>
$C_y=P_{Cy1}$ eqn. 54<br>
$D_y=\mu_y*F_z$ eqn. 55<br>
$\mu_y=(P_{Dy1}+P_{Dy2}*df_z)(1+P_{Dy3}*\gamma^2)$ eqn. 56<br>
$E_y=(P_{Ey1}+P_{Ey2}*df_z)(1+P_{Ey5}*\gamma^2-(P_{Ey3}+P_{Ey4}\gamma)sgn(\alpha))$ eqn. 57<br>
$K_{y\alpha} = P_{Ky1} * F_{z0} * sin(P_{Ky4} * arctan(\frac{F_z}{(P_{Ky2} + P_{Ky5}*\gamma^2) * F_{z0}}))(1-P_{Ky3}|\gamma|)$ eqn. 58<br>
$K_{y\gamma}=F_z(P_{Ky6}+P_{Ky7}*df_z)$ eqn. 59<br>
$B_y=\frac{K_{y\alpha}}{C_y D_y}$ eqn. 60<br>
$S_{Hy}=S_{Hy0}+S_{Hy\gamma}$ eqn. 61<br>
$S_{Hy0}=P_{Hy1}+P_{Hy2}*df_z$ eqn. 62<br>
$S_{Hy\gamma}=\frac{K_{y\gamma}\gamma-S_{Vy\gamma}}{K_{y\alpha}}$ eqn. 63<br>
$S_{Vy}=S_{Vy0}+S_{Vy\gamma}$ eqn. 64<br>
$S_{Vy0}=F_z(P_{Vy1}+P_{Vy2}*df_z)$ eqn. 65<br>
$S_{Vy\gamma}=F_z(P_{Vy3}+P_{Vy4}*df_z)\gamma$ eqn. 66<br>
<br>
And we will plot the final step we got to at the end of the last notebook<br>


In [ ]:
df = remove_time_gaps(filter_vel(cornering, 11.1, 0.1))
df = df[df.TSTC > 50] # only use the runs where the tire is warm, greater than 50 deg C
def model(fz_targ, sa, ia_targ, press, kappa, offsets=False):
    df_z = (fz_targ - fz_nom) / fz_nom
    mu_y = (tire_model.PDY1 + tire_model.PDY2 * df_z) * (1 - tire_model.PDY3 * ia_targ**2)
    kya = tire_model.PKY1 * fz_nom * np.sin(tire_model.PKY4*np.arctan(fz_targ/((tire_model.PKY2 + tire_model.PKY5*(ia_targ**2))*fz_nom))) * (1 - tire_model.PKY3 * np.abs(ia_targ))
    c_y = tire_model.PCY1 # now we get a coefficient for c_y
    d_y = mu_y * fz_targ
    e_y = (tire_model.PEY1 + tire_model.PEY2 * df_z) * (1 + (tire_model.PEY5 * ia_targ**2) - (tire_model.PEY3 + tire_model.PEY4 * ia_targ) * np.sign(sa))# now we get a coefficient for e_y
    b_y = kya / (d_y * c_y) # and then we can get b_y
    shy_0 = tire_model.PHY1 + tire_model.PHY2 * df_z
    svy_0 = (tire_model.PVY1 + tire_model.PVY2 * df_z) * fz_targ
    svy_g = (tire_model.PVY3 + tire_model.PVY4 * df_z) * fz_targ * ia_targ
    svy = svy_0 + svy_g
    kyg = (tire_model.PKY6 + tire_model.PKY7 * df_z) * fz_targ
    shy_g = (kyg * ia_targ - svy_g) / kya
    shy = shy_0 + shy_g
    sa_y = sa + shy
    fy = d_y * np.sin(c_y * np.arctan(b_y * sa_y - e_y * (b_y * sa_y - np.arctan(b_y * sa_y)))) + svy
    if offsets:
        return (fy, np.ones(1) * np.rad2deg(-shy), np.ones(1) * svy)
    return fy
fig = go.Figure()
i, err = 0.0, 0.0
for fza in sweeps:
    for iaa in [0, 2, 4]:
        error = plot_fit(fza, model, slip_angles, np.deg2rad(iaa), pres_nom, 0.0, fig, df, f"Fit {fza} N {iaa} deg")
        if error is not None:
            err += error
            i += 1
print(f"Total err: {(err/i):.2f}")
fig.update_layout(template="plotly_dark", title=f"Fy vs SA", xaxis_title="SA (deg)", yaxis_title="Fy (N)")
fig.update_yaxes(range=[-3000, 3000])
fig.show()

Now lets forget about camber for a moment and look at pressure

In [ ]:
df = remove_time_gaps(filter_vel(cornering, 11.1, 0.1))
df = df[df.TSTC > 50] # only use the runs where the tire is warm, greater than 50 deg C
def model(fz_targ, sa, ia_targ, press, kappa, offsets=False):
    df_z = (fz_targ - fz_nom) / fz_nom
    mu_y = (tire_model.PDY1 + tire_model.PDY2 * df_z) * (1 - tire_model.PDY3 * ia_targ**2)
    kya = tire_model.PKY1 * fz_nom * np.sin(tire_model.PKY4*np.arctan(fz_targ/((tire_model.PKY2 + tire_model.PKY5*(ia_targ**2))*fz_nom))) * (1 - tire_model.PKY3 * np.abs(ia_targ))
    c_y = tire_model.PCY1 # now we get a coefficient for c_y
    d_y = mu_y * fz_targ
    e_y = (tire_model.PEY1 + tire_model.PEY2 * df_z) * (1 + (tire_model.PEY5 * ia_targ**2) - (tire_model.PEY3 + tire_model.PEY4 * ia_targ) * np.sign(sa))# now we get a coefficient for e_y
    b_y = kya / (d_y * c_y) # and then we can get b_y
    shy_0 = tire_model.PHY1 + tire_model.PHY2 * df_z
    svy_0 = (tire_model.PVY1 + tire_model.PVY2 * df_z) * fz_targ
    svy_g = (tire_model.PVY3 + tire_model.PVY4 * df_z) * fz_targ * ia_targ
    svy = svy_0 + svy_g
    kyg = (tire_model.PKY6 + tire_model.PKY7 * df_z) * fz_targ
    shy_g = (kyg * ia_targ - svy_g) / kya
    shy = shy_0 + shy_g
    sa_y = sa + shy
    fy = d_y * np.sin(c_y * np.arctan(b_y * sa_y - e_y * (b_y * sa_y - np.arctan(b_y * sa_y)))) + svy
    if offsets:
        return (fy, np.ones(1) * np.rad2deg(-shy), np.ones(1) * svy)
    return fy
fig = go.Figure()
i, err = 0.0, 0.0
for fza in sweeps:
    for pres in [8*PSI_TO_PA, 10*PSI_TO_PA, 12*PSI_TO_PA, 14*PSI_TO_PA]:
        error = plot_fit(fza, model, slip_angles, np.deg2rad(0.0), pres, 0.0, fig, df, f"Fit {fza} N {pres/1000:.1f} kPa")
        if error is not None:
            err += error
            i += 1
print(f"Total err: {(err/i):.2f}")
fig.update_layout(template="plotly_dark", title=f"Fy vs SA", xaxis_title="SA (deg)", yaxis_title="Fy (N)")
fig.update_yaxes(range=[-3000, 3000])
fig.show()

For the most part the pressure seems to have a very subtle effect, I would have a hard time convincing myself that it mattered at low normal load if I wasn't looking at the RMS error for each of them<br>
We can see that for most normal loads the run at our nominal pressure of 82.7kPa (12psi) has the lowest rms error which is good and that the lower pressure runs have a higher force magnitude through the range of $\alpha$<br>
<br>
Anyway, lets crack on and add some pressure terms<br>
First we need a normalized pressure term<br>
$dp_i=\frac{p_i-p_{i0}}{p_{i0}}$ eqn. 32<br>
And since the main difference appears to be with the max normal force at $\alpha$ saturation we need to add a term to $\mu$<br>
$\mu_y=(P_{Dy1}+P_{Dy2}*df_z)(1+P_{Dy3}*\gamma^2)(1+P_{py3}dp_i+P_{py4}dp_i^2)$

In [ ]:
df = remove_time_gaps(filter_vel(cornering, 11.1, 0.1))
df = df[df.TSTC > 50] # only use the runs where the tire is warm, greater than 50 deg C
def model(fz_targ, sa, ia_targ, press, kappa, offsets=False):
    df_z = (fz_targ - fz_nom) / fz_nom
    dp_i = (press - pres_nom) / pres_nom
    mu_y = (tire_model.PDY1 + tire_model.PDY2 * df_z) * (1 - tire_model.PDY3 * ia_targ**2) * (1 + tire_model.PPY3 * dp_i + tire_model.PPY4 * dp_i**2)
    kya = tire_model.PKY1 * fz_nom * np.sin(tire_model.PKY4*np.arctan(fz_targ/((tire_model.PKY2 + tire_model.PKY5*(ia_targ**2))*fz_nom))) * (1 - tire_model.PKY3 * np.abs(ia_targ))
    c_y = tire_model.PCY1 # now we get a coefficient for c_y
    d_y = mu_y * fz_targ
    e_y = (tire_model.PEY1 + tire_model.PEY2 * df_z) * (1 + (tire_model.PEY5 * ia_targ**2) - (tire_model.PEY3 + tire_model.PEY4 * ia_targ) * np.sign(sa))# now we get a coefficient for e_y
    b_y = kya / (d_y * c_y) # and then we can get b_y
    shy_0 = tire_model.PHY1 + tire_model.PHY2 * df_z
    svy_0 = (tire_model.PVY1 + tire_model.PVY2 * df_z) * fz_targ
    svy_g = (tire_model.PVY3 + tire_model.PVY4 * df_z) * fz_targ * ia_targ
    svy = svy_0 + svy_g
    kyg = (tire_model.PKY6 + tire_model.PKY7 * df_z) * fz_targ
    shy_g = (kyg * ia_targ - svy_g) / kya
    shy = shy_0 + shy_g
    sa_y = sa + shy
    fy = d_y * np.sin(c_y * np.arctan(b_y * sa_y - e_y * (b_y * sa_y - np.arctan(b_y * sa_y)))) + svy
    if offsets:
        return (fy, np.ones(1) * np.rad2deg(-shy), np.ones(1) * svy)
    return fy
fig = go.Figure()
i, err = 0.0, 0.0
for fza in sweeps:
    for pres in [8*PSI_TO_PA, 10*PSI_TO_PA, 12*PSI_TO_PA, 14*PSI_TO_PA]:
        error = plot_fit(fza, model, slip_angles, np.deg2rad(0.0), pres, 0.0, fig, df, f"Fit {fza} N {pres/1000:.1f} kPa")
        if error is not None:
            err += error
            i += 1
print(f"Total err: {(err/i):.2f}")
fig.update_layout(template="plotly_dark", title=f"Fy vs SA", xaxis_title="SA (deg)", yaxis_title="Fy (N)")
fig.update_yaxes(range=[-3000, 3000])
fig.show()

So if we take the average of the rms error (because that was continent to code) for each bin we see a slight improvement, which like whatever. Most of the error is in the asymmetry anyway<br>
Lets add the terms to $K_{y\alpha}$<br>
$K_{y\alpha} = P_{Ky1} * F_{z0} * sin(P_{Ky4} * arctan(\frac{F_z}{(P_{Ky2} + P_{Ky5}*\gamma^2)(1+P_{py2}dp_i)F_{z0}}))(1-P_{Ky3}|\gamma|)$

In [ ]:
df = remove_time_gaps(filter_vel(cornering, 11.1, 0.1))
df = df[df.TSTC > 50] # only use the runs where the tire is warm, greater than 50 deg C
def model(fz_targ, sa, ia_targ, press, kappa, offsets=False):
    df_z = (fz_targ - fz_nom) / fz_nom
    dp_i = (press - pres_nom) / pres_nom
    mu_y = (tire_model.PDY1 + tire_model.PDY2 * df_z) * (1 - tire_model.PDY3 * ia_targ**2) * (1 + tire_model.PPY3 * dp_i + tire_model.PPY4 * dp_i**2)
    kya = tire_model.PKY1 * fz_nom * np.sin(tire_model.PKY4*np.arctan(fz_targ/((tire_model.PKY2 + tire_model.PKY5*(ia_targ**2))*(1 + tire_model.PPY2*dp_i)*fz_nom))) * (1 - tire_model.PKY3 * np.abs(ia_targ))
    c_y = tire_model.PCY1 # now we get a coefficient for c_y
    d_y = mu_y * fz_targ
    e_y = (tire_model.PEY1 + tire_model.PEY2 * df_z) * (1 + (tire_model.PEY5 * ia_targ**2) - (tire_model.PEY3 + tire_model.PEY4 * ia_targ) * np.sign(sa))# now we get a coefficient for e_y
    b_y = kya / (d_y * c_y) # and then we can get b_y
    shy_0 = tire_model.PHY1 + tire_model.PHY2 * df_z
    svy_0 = (tire_model.PVY1 + tire_model.PVY2 * df_z) * fz_targ
    svy_g = (tire_model.PVY3 + tire_model.PVY4 * df_z) * fz_targ * ia_targ
    svy = svy_0 + svy_g
    kyg = (tire_model.PKY6 + tire_model.PKY7 * df_z) * fz_targ
    shy_g = (kyg * ia_targ - svy_g) / kya
    shy = shy_0 + shy_g
    sa_y = sa + shy
    fy = d_y * np.sin(c_y * np.arctan(b_y * sa_y - e_y * (b_y * sa_y - np.arctan(b_y * sa_y)))) + svy
    if offsets:
        return (fy, np.ones(1) * np.rad2deg(-shy), np.ones(1) * svy)
    return fy
fig = go.Figure()
i, err = 0.0, 0.0
for fza in sweeps:
    for pres in [8*PSI_TO_PA, 10*PSI_TO_PA, 12*PSI_TO_PA, 14*PSI_TO_PA]:
        error = plot_fit(fza, model, slip_angles, np.deg2rad(0.0), pres, 0.0, fig, df, f"Fit {fza} N {pres/1000:.1f} kPa")
        if error is not None:
            err += error
            i += 1
print(f"Total err: {(err/i):.2f}")
fig.update_layout(template="plotly_dark", title=f"Fy vs SA", xaxis_title="SA (deg)", yaxis_title="Fy (N)")
fig.update_yaxes(range=[-3000, 3000])
fig.show()

Not clear how much that helped, but the number went down<br>
Now we need to move on to combined IA and pressure<br>

In [ ]:
df = remove_time_gaps(filter_vel(cornering, 11.1, 0.1))
df = df[df.TSTC > 50] # only use the runs where the tire is warm, greater than 50 deg C
def model(fz_targ, sa, ia_targ, press, kappa, offsets=False):
    df_z = (fz_targ - fz_nom) / fz_nom
    dp_i = (press - pres_nom) / pres_nom
    mu_y = (tire_model.PDY1 + tire_model.PDY2 * df_z) * (1 - tire_model.PDY3 * ia_targ**2) * (1 + tire_model.PPY3 * dp_i + tire_model.PPY4 * dp_i**2)
    kya = tire_model.PKY1 * fz_nom * np.sin(tire_model.PKY4*np.arctan(fz_targ/((tire_model.PKY2 + tire_model.PKY5*(ia_targ**2))*(1 + tire_model.PPY2*dp_i)*fz_nom))) * (1 - tire_model.PKY3 * np.abs(ia_targ))
    c_y = tire_model.PCY1 # now we get a coefficient for c_y
    d_y = mu_y * fz_targ
    e_y = (tire_model.PEY1 + tire_model.PEY2 * df_z) * (1 + (tire_model.PEY5 * ia_targ**2) - (tire_model.PEY3 + tire_model.PEY4 * ia_targ) * np.sign(sa))# now we get a coefficient for e_y
    b_y = kya / (d_y * c_y) # and then we can get b_y
    shy_0 = tire_model.PHY1 + tire_model.PHY2 * df_z
    svy_0 = (tire_model.PVY1 + tire_model.PVY2 * df_z) * fz_targ
    svy_g = (tire_model.PVY3 + tire_model.PVY4 * df_z) * fz_targ * ia_targ
    svy = svy_0 + svy_g
    kyg = (tire_model.PKY6 + tire_model.PKY7 * df_z) * fz_targ
    shy_g = (kyg * ia_targ - svy_g) / kya
    shy = shy_0 + shy_g
    sa_y = sa + shy
    fy = d_y * np.sin(c_y * np.arctan(b_y * sa_y - e_y * (b_y * sa_y - np.arctan(b_y * sa_y)))) + svy
    if offsets:
        return (fy, np.ones(1) * np.rad2deg(-shy), np.ones(1) * svy)
    return fy
fig = go.Figure()
i, err = 0.0, 0.0
for fza in sweeps:
    for pres in [8*PSI_TO_PA, 10*PSI_TO_PA, 12*PSI_TO_PA, 14*PSI_TO_PA]:
        for iaa in [0, 2, 4]:
            error = plot_fit(fza, model, slip_angles, np.deg2rad(iaa), pres, 0.0, fig, df, f"Fit {fza} N {iaa} deg {pres/1000:.1f} kPa")
            if error is not None:
                err += error
                i += 1
print(f"Total err: {(err/i):.2f}")
fig.update_layout(template="plotly_dark", title=f"Fy vs SA", xaxis_title="SA (deg)", yaxis_title="Fy (N)")
fig.update_yaxes(range=[-3000, 3000])
fig.show()

At high normal force there is a bit of shift so we shall add in the pressure & ia term<br>
$K_{y\gamma}=F_z(P_{Ky6}+P_{Ky7}*df_z)(1+P_{py5}dp_i)$

In [ ]:
df = remove_time_gaps(filter_vel(cornering, 11.1, 0.1))
df = df[df.TSTC > 50] # only use the runs where the tire is warm, greater than 50 deg C
def model(fz_targ, sa, ia_targ, press, kappa, offsets=False):
    df_z = (fz_targ - fz_nom) / fz_nom
    dp_i = (press - pres_nom) / pres_nom
    mu_y = (tire_model.PDY1 + tire_model.PDY2 * df_z) * (1 - tire_model.PDY3 * ia_targ**2) * (1 + tire_model.PPY3 * dp_i + tire_model.PPY4 * dp_i**2)
    kya = tire_model.PKY1 * fz_nom * np.sin(tire_model.PKY4*np.arctan(fz_targ/((tire_model.PKY2 + tire_model.PKY5*(ia_targ**2))*(1 + tire_model.PPY2*dp_i)*fz_nom))) * (1 - tire_model.PKY3 * np.abs(ia_targ))
    c_y = tire_model.PCY1 # now we get a coefficient for c_y
    d_y = mu_y * fz_targ
    e_y = (tire_model.PEY1 + tire_model.PEY2 * df_z) * (1 + (tire_model.PEY5 * ia_targ**2) - (tire_model.PEY3 + tire_model.PEY4 * ia_targ) * np.sign(sa))# now we get a coefficient for e_y
    b_y = kya / (d_y * c_y) # and then we can get b_y
    shy_0 = tire_model.PHY1 + tire_model.PHY2 * df_z
    svy_0 = (tire_model.PVY1 + tire_model.PVY2 * df_z) * fz_targ
    svy_g = (tire_model.PVY3 + tire_model.PVY4 * df_z) * fz_targ * ia_targ
    svy = svy_0 + svy_g
    kyg = (tire_model.PKY6 + tire_model.PKY7 * df_z) * (1 + tire_model.PPY5 * dp_i) * fz_targ
    shy_g = (kyg * ia_targ - svy_g) / kya
    shy = shy_0 + shy_g
    sa_y = sa + shy
    fy = d_y * np.sin(c_y * np.arctan(b_y * sa_y - e_y * (b_y * sa_y - np.arctan(b_y * sa_y)))) + svy
    if offsets:
        return (fy, np.ones(1) * np.rad2deg(-shy), np.ones(1) * svy)
    return fy
fig = go.Figure()
i, err = 0.0, 0.0
for fza in sweeps:
    for pres in [8*PSI_TO_PA, 10*PSI_TO_PA, 12*PSI_TO_PA, 14*PSI_TO_PA]:
        for iaa in [0, 2, 4]:
            error = plot_fit(fza, model, slip_angles, np.deg2rad(iaa), pres, 0.0, fig, df, f"Fit {fza} N {iaa} deg {pres/1000:.1f} kPa")
            if error is not None:
                err += error
                i += 1
print(f"Total err: {(err/i):.2f}")
fig.update_layout(template="plotly_dark", title=f"Fy vs SA", xaxis_title="SA (deg)", yaxis_title="Fy (N)")
fig.update_yaxes(range=[-3000, 3000])
fig.show()

Little to no change, but were are at this point limited by two factors:<br>
1. The quality of the fit I pre-prepared for this<br>
2. The variation of the tire and just general real world shit<br>
<br>
Pacejka / Besselink / Schmeitz leave in $\lambda$ parameters all over the place that you can scale any part of this equation<br>
To the best of my understanding these parameters exist to fit the model to specific road conditions, individual tires, or even perhaps temperature <br>
For this reason I have left them out of the equations here as we are just fitting to a tire testing machine

Thus far we have only been using the equations for $F_{yp}$, or $F_y$ in *pure slip* (pure slip angle), but we want the force when there is combined slip angle and slip ratio<br>
And for this we get the equation:<br>
$F_y=G_{y\kappa}F_{yp}+S_{Vy\kappa}$<br>
Which scales and vertically shifts the pure slip force to create the combined force<br>
<br>
$G_{y\kappa}=\frac{cos[C_{y\kappa}arctan\{B_{y\kappa}\kappa_s-E_{y\kappa}(B_{y\kappa}\kappa_s-arctan(B_{y\kappa}\kappa_s))\}]}{cos[C_{y\kappa}arctan\{B_{y\kappa}S_{Hy\kappa}-E_{y\kappa}(B_{y\kappa}S_{Hy\kappa}-arctan(B_{y\kappa}S_{Hy\kappa}))\}]}$<br>
And ngl this looks really intimidating<br>
Lets simplify if to reason about it<br>
If $S_{Hy\kappa}=0$ then $G_{y\kappa}=cos[C_{y\kappa}arctan\{B_{y\kappa}\kappa_s-E_{y\kappa}(B_{y\kappa}\kappa_s-arctan(B_{y\kappa}\kappa_s))\}]$<br>
And then $C_{y\kappa}$, $E_{y\kappa}$, and $B_{y\kappa}$ do the same as before<br>
Here are the equations we just ignore $\gamma$ for the moment<br>
For now we will just go with $S_{Hy\kappa}=0$ for the moment<br>
$\kappa_s=\kappa$<br>
$B_{y\kappa}=r_{By1}cos\{arctan[r_{By2}(\alpha-r_{By3})]\}$<br>
$C_{y\kappa}=r_{Cy1}$<br>
$E_{y\kappa}=r_{Ey1} + r_{Ey2}df_z$<br>


In [ ]:
df = remove_time_gaps(filter_vel(drive_brake, 11.1, 0.1))
df = df[df.TSTC > 50] # only use the runs where the tire is warm, greater than 50 deg C
def model(fz_targ, sa, ia_targ, press, kappa, offsets=False):
    df_z = (fz_targ - fz_nom) / fz_nom
    dp_i = (press - pres_nom) / pres_nom
    mu_y = (tire_model.PDY1 + tire_model.PDY2 * df_z) * (1 - tire_model.PDY3 * ia_targ**2) * (1 + tire_model.PPY3 * dp_i + tire_model.PPY4 * dp_i**2)
    kya = tire_model.PKY1 * fz_nom * np.sin(tire_model.PKY4*np.arctan(fz_targ/((tire_model.PKY2 + tire_model.PKY5*(ia_targ**2))*(1 + tire_model.PPY2*dp_i)*fz_nom))) * (1 - tire_model.PKY3 * np.abs(ia_targ))
    c_y = tire_model.PCY1 # now we get a coefficient for c_y
    d_y = mu_y * fz_targ
    e_y = (tire_model.PEY1 + tire_model.PEY2 * df_z) * (1 + (tire_model.PEY5 * ia_targ**2) - (tire_model.PEY3 + tire_model.PEY4 * ia_targ) * np.sign(sa))# now we get a coefficient for e_y
    b_y = kya / (d_y * c_y) # and then we can get b_y
    shy_0 = tire_model.PHY1 + tire_model.PHY2 * df_z
    svy_0 = (tire_model.PVY1 + tire_model.PVY2 * df_z) * fz_targ
    svy_g = (tire_model.PVY3 + tire_model.PVY4 * df_z) * fz_targ * ia_targ
    svy = svy_0 + svy_g
    kyg = (tire_model.PKY6 + tire_model.PKY7 * df_z) * (1 + tire_model.PPY5 * dp_i) * fz_targ
    shy_g = (kyg * ia_targ - svy_g) / kya
    shy = shy_0 + shy_g
    sa_y = sa + shy
    fy = d_y * np.sin(c_y * np.arctan(b_y * sa_y - e_y * (b_y * sa_y - np.arctan(b_y * sa_y)))) + svy * np.ones(kappa.shape)
    if offsets:
        return (fy, np.ones(1) * np.rad2deg(-shy), np.ones(1) * svy)
    return fy
fig = go.Figure()
i, err = 0.0, 0.0
for fza in sweeps:
    for saa in [0, 3, 6]:
        error = plot_fit(fza, model, np.deg2rad(saa), np.deg2rad(0.0), pres_nom, kappa_sweep, fig, df, f"Fit {fza} N {saa} deg", long=True)
        if error is not None:
            err += error
            i += 1
print(f"Total err: {(err/i):.2f}")
fig.update_layout(template="plotly_dark", title=f"Fy vs SL", xaxis_title="SL", yaxis_title="Fy (N)")
fig.update_yaxes(range=[-3000, 3000])
fig.show()

So thats basically what I expected

In [ ]:
df = remove_time_gaps(filter_vel(drive_brake, 11.1, 0.1))
df = df[df.TSTC > 50] # only use the runs where the tire is warm, greater than 50 deg C
def model(fz_targ, sa, ia_targ, press, kappa, offsets=False):
    df_z = (fz_targ - fz_nom) / fz_nom
    dp_i = (press - pres_nom) / pres_nom
    mu_y = (tire_model.PDY1 + tire_model.PDY2 * df_z) * (1 - tire_model.PDY3 * ia_targ**2) * (1 + tire_model.PPY3 * dp_i + tire_model.PPY4 * dp_i**2)
    kya = tire_model.PKY1 * fz_nom * np.sin(tire_model.PKY4*np.arctan(fz_targ/((tire_model.PKY2 + tire_model.PKY5*(ia_targ**2))*(1 + tire_model.PPY2*dp_i)*fz_nom))) * (1 - tire_model.PKY3 * np.abs(ia_targ))
    c_y = tire_model.PCY1 # now we get a coefficient for c_y
    d_y = mu_y * fz_targ
    e_y = (tire_model.PEY1 + tire_model.PEY2 * df_z) * (1 + (tire_model.PEY5 * ia_targ**2) - (tire_model.PEY3 + tire_model.PEY4 * ia_targ) * np.sign(sa))# now we get a coefficient for e_y
    b_y = kya / (d_y * c_y) # and then we can get b_y
    shy_0 = tire_model.PHY1 + tire_model.PHY2 * df_z
    svy_0 = (tire_model.PVY1 + tire_model.PVY2 * df_z) * fz_targ
    svy_g = (tire_model.PVY3 + tire_model.PVY4 * df_z) * fz_targ * ia_targ
    svy = svy_0 + svy_g
    kyg = (tire_model.PKY6 + tire_model.PKY7 * df_z) * (1 + tire_model.PPY5 * dp_i) * fz_targ
    shy_g = (kyg * ia_targ - svy_g) / kya
    shy = shy_0 + shy_g
    sa_y = sa + shy
    fy_p = d_y * np.sin(c_y * np.arctan(b_y * sa_y - e_y * (b_y * sa_y - np.arctan(b_y * sa_y)))) + svy
    byk = tire_model.RBY1 * np.cos(np.arctan(tire_model.RBY2 * (sa - tire_model.RBY3)))
    cyk = tire_model.RCY1
    eyk = tire_model.REY1 + tire_model.REY2 * df_z
    if offsets:
        print(eyk)
    kappa_s = kappa
    shyk = 0
    gyk0 = np.cos(cyk * np.arctan(byk * shyk - eyk * (byk * shyk - np.arctan(byk * shyk))))
    gyk = np.cos(cyk * np.arctan(byk * kappa_s - eyk * (byk * kappa_s - np.arctan(byk * kappa_s)))) / gyk0
    fy = fy_p * gyk
    if offsets:
        return (fy, np.ones(1) * shyk, np.ones(1) * 0)
    return fy
fig = go.Figure()
i, err = 0.0, 0.0
for fza in sweeps:
    for saa in [0, 3, 6]:
        error = plot_fit(fza, model, np.deg2rad(saa), np.deg2rad(0.0), pres_nom, kappa_sweep, fig, df, f"Fit {fza} N {saa} deg", long=True)
        if error is not None:
            err += error
            i += 1
print(f"Total err: {(err/i):.2f}")
fig.update_layout(template="plotly_dark", title=f"Fy vs SL", xaxis_title="SL", yaxis_title="Fy (N)")
fig.show()

In [ ]:
print(tire_model.REY1)
print(tire_model.REY2)

Right now this is pretty dogshit, but thats all because of my pre-prepared fit<br>
Its way to compressed along kappa, lets go and add $S_{Hy\kappa}$ now<br>
$S_{Hy\kappa}=r_{Hy1}+r_{Hy2}df_z$ 

In [ ]:
df = remove_time_gaps(filter_vel(drive_brake, 11.1, 0.1))
df = df[df.TSTC > 50] # only use the runs where the tire is warm, greater than 50 deg C
def model(fz_targ, sa, ia_targ, press, kappa, offsets=False):
    df_z = (fz_targ - fz_nom) / fz_nom
    dp_i = (press - pres_nom) / pres_nom
    mu_y = (tire_model.PDY1 + tire_model.PDY2 * df_z) * (1 - tire_model.PDY3 * ia_targ**2) * (1 + tire_model.PPY3 * dp_i + tire_model.PPY4 * dp_i**2)
    kya = tire_model.PKY1 * fz_nom * np.sin(tire_model.PKY4*np.arctan(fz_targ/((tire_model.PKY2 + tire_model.PKY5*(ia_targ**2))*(1 + tire_model.PPY2*dp_i)*fz_nom))) * (1 - tire_model.PKY3 * np.abs(ia_targ))
    c_y = tire_model.PCY1 # now we get a coefficient for c_y
    d_y = mu_y * fz_targ
    e_y = (tire_model.PEY1 + tire_model.PEY2 * df_z) * (1 + (tire_model.PEY5 * ia_targ**2) - (tire_model.PEY3 + tire_model.PEY4 * ia_targ) * np.sign(sa))# now we get a coefficient for e_y
    b_y = kya / (d_y * c_y) # and then we can get b_y
    shy_0 = tire_model.PHY1 + tire_model.PHY2 * df_z
    svy_0 = (tire_model.PVY1 + tire_model.PVY2 * df_z) * fz_targ
    svy_g = (tire_model.PVY3 + tire_model.PVY4 * df_z) * fz_targ * ia_targ
    svy = svy_0 + svy_g
    kyg = (tire_model.PKY6 + tire_model.PKY7 * df_z) * (1 + tire_model.PPY5 * dp_i) * fz_targ
    shy_g = (kyg * ia_targ - svy_g) / kya
    shy = shy_0 + shy_g
    sa_y = sa + shy
    fy_p = d_y * np.sin(c_y * np.arctan(b_y * sa_y - e_y * (b_y * sa_y - np.arctan(b_y * sa_y)))) + svy
    byk = tire_model.RBY1 * np.cos(np.arctan(tire_model.RBY2 * (sa - tire_model.RBY3)))
    cyk = tire_model.RCY1
    eyk = tire_model.REY1 + tire_model.REY2 * df_z
    kappa_s = kappa
    shyk = tire_model.RHY1 + tire_model.RHY2 * df_z
    gyk = np.cos(cyk * np.arctan(byk * kappa_s - eyk * (byk * kappa_s - np.arctan(byk * kappa_s)))) / np.cos(cyk * np.arctan(byk * shyk - eyk * (byk * shyk - np.arctan(byk * shyk))))
    fy = fy_p * gyk
    if offsets:
        return (fy, np.ones(1) * shyk, np.ones(1) * 0)
    return fy
fig = go.Figure()
i, err = 0.0, 0.0
for fza in sweeps:
    for saa in [0, 3, 6]:
        error = plot_fit(fza, model, np.deg2rad(saa), np.deg2rad(0.0), pres_nom, kappa_sweep, fig, df, f"Fit {fza} N {saa} deg", long=True)
        if error is not None:
            err += error
            i += 1
print(f"Total err: {(err/i):.2f}")
fig.update_layout(template="plotly_dark", title=f"Fy vs SL", xaxis_title="SL", yaxis_title="Fy (N)")
fig.show()

That didn't do anything lol<br>
Now we do $S_{Vy\kappa}$<br>
$S_{Vy\kappa}=D_{Vy\kappa}sin(r_{Vy5}arctan(r_{Vy6}\kappa))$ <br>
$D_{Vy\kappa}=\mu_yF_z(r_{Vy1}+r_{Vy2}df_z)cos(arctan(r_{Vy4}\alpha_F))$

In [ ]:
df = remove_time_gaps(filter_vel(drive_brake, 11.1, 0.1))
df = df[df.TSTC > 50] # only use the runs where the tire is warm, greater than 50 deg C
def model(fz_targ, sa, ia_targ, press, kappa, offsets=False):
    df_z = (fz_targ - fz_nom) / fz_nom
    dp_i = (press - pres_nom) / pres_nom
    mu_y = (tire_model.PDY1 + tire_model.PDY2 * df_z) * (1 - tire_model.PDY3 * ia_targ**2) * (1 + tire_model.PPY3 * dp_i + tire_model.PPY4 * dp_i**2)
    kya = tire_model.PKY1 * fz_nom * np.sin(tire_model.PKY4*np.arctan(fz_targ/((tire_model.PKY2 + tire_model.PKY5*(ia_targ**2))*(1 + tire_model.PPY2*dp_i)*fz_nom))) * (1 - tire_model.PKY3 * np.abs(ia_targ))
    c_y = tire_model.PCY1 # now we get a coefficient for c_y
    d_y = mu_y * fz_targ
    e_y = (tire_model.PEY1 + tire_model.PEY2 * df_z) * (1 + (tire_model.PEY5 * ia_targ**2) - (tire_model.PEY3 + tire_model.PEY4 * ia_targ) * np.sign(sa))# now we get a coefficient for e_y
    b_y = kya / (d_y * c_y) # and then we can get b_y
    shy_0 = tire_model.PHY1 + tire_model.PHY2 * df_z
    svy_0 = (tire_model.PVY1 + tire_model.PVY2 * df_z) * fz_targ
    svy_g = (tire_model.PVY3 + tire_model.PVY4 * df_z) * fz_targ * ia_targ
    svy = svy_0 + svy_g
    kyg = (tire_model.PKY6 + tire_model.PKY7 * df_z) * (1 + tire_model.PPY5 * dp_i) * fz_targ
    shy_g = (kyg * ia_targ - svy_g) / kya
    shy = shy_0 + shy_g
    sa_y = sa + shy
    fy_p = d_y * np.sin(c_y * np.arctan(b_y * sa_y - e_y * (b_y * sa_y - np.arctan(b_y * sa_y)))) + svy
    byk = tire_model.RBY1 * np.cos(np.arctan(tire_model.RBY2 * (sa - tire_model.RBY3)))
    cyk = tire_model.RCY1
    eyk = tire_model.REY1 + tire_model.REY2 * df_z
    kappa_s = kappa
    shyk = tire_model.RHY1 + tire_model.RHY2 * df_z
    gyk0 = np.cos(cyk * np.arctan(byk * shyk - eyk * (byk * shyk - np.arctan(byk * shyk))))
    gyk = np.cos(cyk * np.arctan(byk * kappa_s - eyk * (byk * kappa_s - np.arctan(byk * kappa_s)))) / gyk0
    dvyk = mu_y * fz_targ * (tire_model.RVY1 + tire_model.RVY2 * df_z) * np.cos(np.arctan(sa * tire_model.RVY4))
    svyk = dvyk * np.sin(tire_model.RVY5 * np.arctan(tire_model.RVY6 * kappa))
    fy = fy_p * gyk + svyk
    if offsets:
        return (fy, np.ones(1) * shyk, np.ones(1) * svyk)
    return fy
fig = go.Figure()
i, err = 0.0, 0.0
for fza in sweeps:
    for saa in [0, 3, 6]:
        error = plot_fit(fza, model, np.deg2rad(saa), np.deg2rad(0.0), pres_nom, kappa_sweep, fig, df, f"Fit {fza} N {saa} deg", long=True)
        if error is not None:
            err += error
            i += 1
print(f"Total err: {(err/i):.2f}")
fig.update_layout(template="plotly_dark", title=f"Fy vs SL", xaxis_title="SL", yaxis_title="Fy (N)")
fig.show()